In [ ]:
!pip install paho-mqtt
!pip install influxdb-client

In [13]:
import paho.mqtt.client as mqtt
import time
import json
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS


# Configurações do Broker MQTT
MQTT_BROKER = "broker.hivemq.com"
MQTT_PORT = 8883  # Porta para conexão TLS/SSL
MQTT_USER = "teste"  # Insira seu nome de usuário aqui
MQTT_PASSWORD = "TESTEteste@2024"  # Insira sua senha aqui
MQTT_TOPIC = "teste_topico"

# Configurações do InfluxDB
INFLUXDB_URL = "https://us-east-1-1.aws.cloud2.influxdata.com"
INFLUXDB_TOKEN = "s5TjFcL80railpA3GKRqVvnT5iX-7uxsPAfshrH_6GRK0Elem_QwIomCOgdRwEyGusSAvSecReWLOuTrvM4E4w=="
INFLUXDB_ORG = "Uninove"
INFLUXDB_BUCKET = "Sensores"

# Conectar ao InfluxDB
influxdb_client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
write_api = influxdb_client.write_api(write_options=SYNCHRONOUS)

# Variável para armazenar a última mensagem recebida
last_message = None

# Callback quando a conexão é estabelecida
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT Broker!")
        client.subscribe(MQTT_TOPIC)  # Inscreve-se no tópico onde os dados estão sendo publicados
    else:
        print(f"Failed to connect, return code {rc}")

# Callback quando uma mensagem é recebida
def on_message(client, userdata, msg):
    global last_message
    last_message = msg.payload.decode()  # Atualiza a última mensagem recebida
    print(f"Received message from topic {MQTT_TOPIC}: {last_message}")
    process_message(last_message)

# Configuração do cliente MQTT
client = mqtt.Client()
client.username_pw_set(MQTT_USER, MQTT_PASSWORD)
client.tls_set()  # Usar TLS para a conexão segura

client.on_connect = on_connect
client.on_message = on_message

# Conectar ao broker MQTT
client.connect(MQTT_BROKER, MQTT_PORT)

# Função para processar a mensagem recebida
def process_message(message):
    try:
        data = json.loads(message)
        temperature = data['temperature']
        humidity = data['humidity']
        store_in_influxdb(temperature=temperature, humidity=humidity)
    except json.JSONDecodeError:
        print("Failed to decode JSON from message.")
    except KeyError as e:
        print(f"Missing expected data in message: {e}")

# Função para armazenar os dados no InfluxDB
def store_in_influxdb(temperature, humidity):
    # Convertendo os valores float para int
    temperature_int = int(temperature)
    humidity_int = int(humidity)

    point = Point("sensor_dht22")\
        .field("temperature", temperature_int)\
        .field("humidity", humidity_int)
    write_api.write(bucket=INFLUXDB_BUCKET, org=INFLUXDB_ORG, record=point)
    print("Dados Inseridos no InfluxDB com Sucesso!")

# Loop de leitura com intervalo de 2 segundos
client.loop_start()  # Inicia o loop MQTT em segundo plano

try:
    while True:
        time.sleep(2)  # Aguarda 2 segundos antes de processar a próxima mensagem
except KeyboardInterrupt:
    print("Exiting...")
finally:
    client.loop_stop()  # Para o loop MQTT
    client.disconnect()  # Desconecta do broker

<ipython-input-13-dde17c811d4d>:44: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Connected to MQTT Broker!
Received message from topic teste_topico: {"humidity": 98.5, "temperature": 80.0}
Dados Inseridos no InfluxDB com Sucesso!
Received message from topic teste_topico: {"humidity": 98.5, "temperature": 80.0}
Dados Inseridos no InfluxDB com Sucesso!
Received message from topic teste_topico: {"humidity": 98.5, "temperature": 80.0}
Dados Inseridos no InfluxDB com Sucesso!
Received message from topic teste_topico: {"humidity": 98.5, "temperature": 80.0}
Dados Inseridos no InfluxDB com Sucesso!
Received message from topic teste_topico: {"humidity": 98.5, "temperature": 80.0}
Dados Inseridos no InfluxDB com Sucesso!
Received message from topic teste_topico: {"humidity": 98.5, "temperature": 80.0}
Dados Inseridos no InfluxDB com Sucesso!
Received message from topic teste_topico: {"humidity": 98.5, "temperature": 80.0}
Dados Inseridos no InfluxDB com Sucesso!
Exiting...
